<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

# ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Kafka Consumption with Explicit Deserializer

Consume the topic named `ratings`

## ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) General Set-up

In [ ]:
from confluent_kafka import Consumer, KafkaError
from confluent_kafka import avro
from confluent_kafka.avro import AvroConsumer
from confluent_kafka.avro.serializer import SerializerError
import json
import qcutils
import time

servers=qcutils.read_config_value("kafka.server") + ":" + str(qcutils.read_config_value("kafka.port"))
sr_url=qcutils.read_config_value("kafka.schema_registry.url")

print(servers)
print(sr_url)

## ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) Avro Consumer (`ratings` Topic)

#### Check for schemas in the Schema Registry

In [ ]:
from confluent_kafka import DeserializingConsumer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroDeserializer
from confluent_kafka.serialization import StringDeserializer

sr_conf = {'url': sr_url}
schema_registry_client = SchemaRegistryClient(sr_conf)

schema_registry_client.get_subjects()

#### Get the latest version of the schema of your interest

Note that, for each topic, the key and value schema are named as <topic-name>-key and <topic-name>-value.

As usual, it is not mandatory to have the key schema for an avro topic. By default, the key for an avro topic is a String.
    
In particular, the `ratings` topic have the value schema ('ratings-value'), but not the key schema.

In [ ]:
value_schema = schema_registry_client.get_latest_version('ratings-value').schema.schema_str
print(value_schema)

#### Let's consume the topic using the right serializers

In [ ]:
topic = 'ratings'
consumer_group = 'MB-CG'

key_deserializer = StringDeserializer()
value_deserializer = AvroDeserializer(schema_str=value_schema,schema_registry_client=schema_registry_client)


consumer_conf = {'bootstrap.servers': servers,
                 'key.deserializer': key_deserializer,
                 'value.deserializer': value_deserializer,
                 'group.id': consumer_group,
                 'auto.offset.reset': 'latest'}

c = DeserializingConsumer(consumer_conf)
c.subscribe([topic])

waiting = False

try:
    while True:
        msg = c.poll(1.0)
        if msg is None:
            if waiting:
                print(".",end =" ")
            else:
                print("Waiting",end =" ")
                waiting = True
            continue
        elif msg.error():
            print('error: {}'.format(msg.error()))
            waiting = False
        else:
            value = msg.value()
            key = msg.key()
            print("\nConsumed record with key {} and value {}".format(key, value))
            waiting = False
except KeyboardInterrupt:
    pass
finally:
    c.close()  

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.